In [ ]:
import spacy
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from pickle import dump,load

In [ ]:
nlp=spacy.load("en_core_web_sm",disable=['parser','tagger','ner'])

In [ ]:
def read_book(path):
    with open(path,'r') as f:
        book=f.read()
    return book


In [ ]:
#This function creates tokens by ignoring the special charecters
def seperate_punctation(text):
    return [token.text.lower() for token in nlp(text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']


In [ ]:
#replace by the length of the tokens
nlp.max_length=1198623

In [ ]:
# Read the book and convert into tokens
book=read_book("/content/drive/MyDrive/DataSets/moby_dick_four_chapters.txt")
tokens=seperate_punctation(book)

In [ ]:
# Creates sequences of text
len_sequences=25+1
text_sequences=[]
for i in range(len_sequences,len(tokens)-len_sequences):
    text_sequences.append(tokens[i:i+len_sequences])

In [ ]:
#Tokenizer gets adjusted according to the  sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [ ]:
sequences=tokenizer.texts_to_sequences(text_sequences)

In [ ]:
sequences=np.array(sequences)
vocabulary_size=len(tokenizer.word_counts)

In [ ]:
x=sequences[:,:-1]
y=sequences[:,-1]
y=to_categorical(y,num_classes=vocabulary_size+1)
seq_len=x.shape[1]

In [ ]:

def create_model(vocabulary_size, seq_len):
    model = Sequential([
        Embedding(vocabulary_size, 25),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(50, activation="relu"),
        Dense(vocabulary_size, activation='softmax'),
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Assuming vocabulary_size and seq_len are defined
model = create_model(vocabulary_size=vocabulary_size + 1, seq_len=seq_len)
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x,y,batch_size=128,epochs=300,verbose=True)

Epoch 1/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 80ms/step - accuracy: 0.0414 - loss: 7.4409
Epoch 2/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.0518 - loss: 6.3847
Epoch 3/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.0515 - loss: 6.3526
Epoch 4/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.0498 - loss: 6.2462
Epoch 5/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.0512 - loss: 6.1353
Epoch 6/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.0536 - loss: 6.0178
Epoch 7/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.0550 - loss: 5.9114
Epoch 8/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.0597 - loss: 5.8526
Epoch 9/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.0600 - loss: 5.7969
Epoch 10/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.0696 - loss: 5.7388
Epoch 11/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.0635 - loss: 5.6637
Epoch 12/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/s

In [ ]:
model.save("/content/drive/MyDrive/DataSets/chatbot_model.h5")

In [ ]:
dump(tokenizer,open("/content/drive/MyDrive/DataSets/tokenizer_chatbot",'wb'))

In [ ]:

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = np.argmax(model.predict(pad_encoded, verbose=0), axis=-1)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' ' + pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)



In [ ]:
import random
random_pick=random.randint(0,len(text_sequences))
random_seed_text=text_sequences[random_pick]
seed_text=" ".join(random_seed_text)
seed_text

'frost is all on the outside or whether thou observest it from that sashless window where the frost is on both sides and of which the'

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=10)

In [ ]:
seed_text=input("The story line Which needs to be continued:")

In [ ]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=10)

'it with the middle of the dreary street shouldering i'